Given an input string (s) and a pattern (p), implement wildcard pattern matching with support for '?' and '*' where:

- '?' Matches any single character.
- '*' Matches any sequence of characters (including the empty sequence).
The matching should cover the entire input string (not partial).

 
Example 1:

- Input: s = "aa", p = "a"
- Output: false
- Explanation: "a" does not match the entire string "aa".

Example 2:

- Input: s = "aa", p = "*"
- Output: true
- Explanation: '*' matches any sequence.

Example 3:

- Input: s = "cb", p = "?a"
- Output: false
- Explanation: '?' matches 'c', but the second letter is 'a', which does not match 'b'.


In [29]:
# this is basically the same as question 10, but with a different method of wildcard matching.

# We are going to implement the same thing but using recursive dynamic programming with memoization. Basically it's going to use a dict of int tuples as the key instead of a string tuple. This will allow us to avoid the overhead of creating a string tuple for every recursive call and will make the code a bit faster and more compute/memory efficient

# set recursion limit to 10^6
import sys
sys.setrecursionlimit(10**6)

class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        memo = {}
        ns = len(s)
        np = len(p)
    
        # i is the current position in the string s
        # j is the current position in the pattern p
        def dp(i: int, j: int) -> bool:
            #print(f"i = {i}, j = {j}")
            if (i, j) in memo:
                return memo[(i, j)]
            
            # check for the existence of a wildcard
            hasWildcard = j < np and p[j] == "*"
            
            # if we've reached the end of the pattern, we must 
            # also be at the end of the string
            if j >= np:
                result = i >= ns
                memo[(i,j)] = result
                return result
            
            # if we've reached the end of the string, we need to 
            # either be at the end of the pattern or have a wildcard
            if i >= ns:
                result = hasWildcard and dp(i, j + 1)
                memo[(i,j)] = result
                return result
            
            # check if there is a possible match
            firstMatch = i < ns and (p[j] == "?" or s[i] == p[j])

            if hasWildcard:
                # either we don't use the wildcard (skip it) 
                # or use it at least once and try to match more
                result = dp(i, j + 1) or dp(i + 1, j)
            else:
                # match the current character, consume it and move on
                result = firstMatch and dp(i+1, j+1)
            
            memo[(i, j)] = result
            return result
        
        return dp(0, 0)

In [ ]:
# similar to the above solution, but with a different way of handling 
# the wildcard with lookahead. this one is actually slower than the 
# one above

class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        memo = {}
        ns = len(s)
        np = len(p)
    
        # i is the current position in the string s
        # j is the current position in the pattern p
        def dp(i: int, j: int) -> bool:
            if (i, j) in memo:
                return memo[(i, j)]
            
            # If we've reached the end of the pattern
            if j >= np:
                result = i >= ns
                memo[(i,j)] = result
                return result
            
            # If we have a wildcard
            if p[j] == '*':
                # We can either skip this wildcard entirely (don't 
                # use it) or use it to consume the current character 
                # and keep the wildcard. We can optimize by looking 
                # ahead to see if we can match the rest of the pattern
                if j + 1 >= np:
                    # If * is the last character, it can match 
                    # everything remaining
                    result = True
                else:
                    # Attempt to match the remaining pattern at each 
                    # possible position. Start with not using the * 
                    # at all
                    result = dp(i, j + 1)
                    # Then try matching at each remaining position, 
                    # but only if we haven't found a match yet
                    k = i
                    while not result and k < ns:
                        result = dp(k + 1, j + 1)
                        k += 1
            else:
                # Not a wildcard so we must match current character
                if i >= ns:
                    result = False
                else:
                    firstMatch = p[j] == '?' or s[i] == p[j]
                    result = firstMatch and dp(i + 1, j + 1)
            
            memo[(i,j)] = result
            return result
        
        return dp(0, 0)

In [ ]:
# not my solution, but one that uses greedy matching and backtracking. It's like 50x faster than the above solutions

class Solution:
    def isMatch(self, s: str, p: str) -> bool:
        i, j = 0, 0
        last_match, star = 0, -1
        ns, np = len(s), len(p)

        while i < ns:
            if j < np and (s[i] == p[j] or p[j] == '?'):
                i += 1
                j += 1
            elif j < np and p[j] == '*':
                last_match = i
                star = j
                j += 1
            elif star != -1:
                j = star + 1
                i = last_match + 1
                last_match += 1
            else:
                return False

        while j < np and p[j] == '*':
            j += 1

        return j == np

In [ ]:
s = "aa"
p = "a"
result = Solution().isMatch(s, p)
print(f"result: {result}, expected: False")

s = "aa"
p = "*"
result = Solution().isMatch(s, p)
print(f"result: {result}, expected: True")

s = "cb"
p = "?a"
result = Solution().isMatch(s, p)
print(f"result: {result}, expected: False")

s = "cb"
p = "c?"
result = Solution().isMatch(s, p)
print(f"result: {result}, expected: True")

s = "acdcb"
p = "a*c?b"
result = Solution().isMatch(s, p)
print(f"result: {result}, expected: False")

result: False, expected: False
result: True, expected: True
result: False, expected: False
result: True, expected: True
result: False, expected: False
